In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sn
import folium
import math
import json
from branca.colormap import LinearColormap
from branca.element import MacroElement
from jinja2 import Template
import matplotlib.pyplot as plt

In [2]:
mentions = pd.read_csv('../Data_sinan/mentions.csv',\
                              dtype={'EventRootCode': np.str} \
                             )

#protests_df = pd.read_csv('../Data_sinan/Protests.csv')

country_info_df = pd.read_csv('../Data_sinan/countryinfo.csv')
world_borders = json.load(open('../Data/world_borders.topojson.json'))

fips_codes = pd.read_csv('../Data/fipscountries.csv')
fips_codes = fips_codes[['fips','country']]

happiness_df = pd.read_excel('../Data_sinan/WHR2018Chapter2OnlineData.xls', sheet_name='Figure2.2')
happiness_df = happiness_df[['Country','Happiness score']]
happiness_df.rename(index=str, columns={"Country": "country"}, inplace=True)

politics_df = pd.read_excel('../Data_sinan/WHR2018Chapter2OnlineData.xls')
politics_df = politics_df[politics_df['year']==2016][['country','Confidence in national government', 'Perceptions of corruption']]
politics_df = politics_df.dropna()

democracy_index_df =  pd.read_csv('../Data_sinan/democracy_index.csv')
democracy_index_df =  democracy_index_df[['Country', 'Score']]
democracy_index_df.rename(index=str, columns={"Country": "country", "Score": "Democratic Quality"}, inplace=True)

In [3]:
mentions.head()

,LocEvent,EventRootCode,NumberOfMentions
0,RS,--,4
1,JO,--,3
2,JA,--,7
3,NaN,--,14
4,ES,--,4


In [4]:
#mentions['EventRootCode'].value_counts()

In [5]:
mentions = (mentions[(mentions["EventRootCode"]!='X') & (mentions["EventRootCode"]!='--')])

In [6]:
#test['NumberOfInterMentions']= test.apply(lambda x : x['NumberOfInterMentions']/total_mention[x['LocMention']], axis = 1)

#Summing the number of mentions by country
mentions_by_country = mentions.groupby(['LocEvent'], as_index=False).sum()[['LocEvent','NumberOfMentions']]

#Summing the number of protest mentions by country
df = mentions.groupby(['LocEvent', 'EventRootCode'], as_index=False).sum()
protest_mentions_by_country = df[df['EventRootCode']=='14'].drop(columns=['EventRootCode']) \
                                                           .rename(index=str, 
                                                                   columns={"NumberOfMentions": "Number of protest mentions"},\
                                                                  )


relative_mentions = protest_mentions_by_country.merge(mentions_by_country, on='LocEvent', how='inner')
relative_mentions['Relative number of protest'] = relative_mentions['Number of protest mentions']/relative_mentions['NumberOfMentions']





In [7]:
relative_mentions.head()

,LocEvent,Number of protest mentions,NumberOfMentions,Relative number of protest
0,AA,203,47695,0.004256
1,AC,216,98264,0.002198
2,AE,11925,2934690,0.004063
3,AF,56085,9718130,0.005771
4,AG,15104,1456706,0.010369


In [8]:
relative_mentions = fips_codes.merge(relative_mentions, \
                                     left_on='fips', \
                                     right_on='LocEvent', \
                                     how='inner' \
                                    ) \
                              .drop(columns=['LocEvent'])

In [9]:
population = country_info_df[['fips', 'population']]
relative_mentions = relative_mentions.merge(population, on='fips', how='inner')
relative_mentions = relative_mentions.merge(happiness_df, on='country', how='inner')
relative_mentions = relative_mentions.merge(politics_df, on='country', how='inner')
relative_mentions = relative_mentions.merge(democracy_index_df, on='country', how='inner')


In [10]:
relative_mentions.head()

,fips,country,Number of protest mentions,NumberOfMentions,Relative number of protest,population,Happiness score,Confidence in national government,Perceptions of corruption,Democratic Quality
0,AF,Afghanistan,56085,9718130,0.005771,29121286,3.6315,0.324990,0.793246,2.55
1,AL,Albania,62138,3367533,0.018452,2986952,4.5860,0.400910,0.901071,5.98
2,AM,Armenia,42372,2679371,0.015814,2968000,4.3206,0.184713,0.921421,4.11
3,AR,Argentina,49026,5527104,0.008870,41343201,6.3880,0.419562,0.850924,6.96
4,AU,Austria,22190,3175294,0.006988,8205000,7.1394,0.432638,0.523641,8.42


In [11]:
corr = relative_mentions[['Relative number of protest', 'Confidence in national government', \
                          'Democratic Quality', 'Happiness score', 'Perceptions of corruption']].corr()
corr.style.background_gradient()

,Relative number of protest,Confidence in national government,Democratic Quality,Happiness score,Perceptions of corruption
Relative number of protest,1,-0.157851,-0.263681,-0.16182,0.220258
Confidence in national government,-0.157851,1,-0.240156,-0.237677,-0.389842
Democratic Quality,-0.263681,-0.240156,1,0.730701,-0.370514
Happiness score,-0.16182,-0.237677,0.730701,1,-0.437805
Perceptions of corruption,0.220258,-0.389842,-0.370514,-0.437805,1


In [12]:
relative_mentions.shape

(119, 10)